# Realign two EEGLab.Epoch object's epochs.

@Author [@FranckPrts](https://github.com/FranckPrts).

Here we attempt to provide a python-based solution to the following question: **How to can I realign concurrent epochs from two epoched object?**

Our main goal here is to be able to target all epochs in a given EEG that doesn't have a *sister* (concurently recorded) epoch in the other EEG because it was rejected by preprocessing. At the end, we should have two epoched EEG with the same amount of epochs and where each pair of epoch has their *sister* epoch at the same index.

Fist, the EEG data of each participant was segmented in 1sec epochs before moving on to preprocessing independelty each two EEG data. For each stage of this iterative process (in our case 2-3 iterations) the ID of the rejected epochs were noted in a separate file.

Our issue arise from the fact that once each step was performed, saving the data would lead to losing track of what was the epochs original IDs. 
As exemplified below: 

#IMAGE

In that process we see that an epoch that originally had the ID #6 can end up with the new ID #3. 

To retrieve the original id of the epoch, we will have to work bakward from the last iteration of preprocessing to the first iteration. At each step we will store what was the previous ID of the epochs so we can find their original IDs. 

## Imports

In [1]:
import utils

We import two eeg stream that were preprocessed in MATLAB

In [ ]:
eeg1 = 1
eeg2 = 2